## Практическое задание

<ol>
    <li>Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.
    </li>
    <li>Описать также в анализе какие необоходимо внести изменения  в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.
    </li>
</ol>

In [0]:
from tensorflow.keras import regularizers

In [0]:
import tensorflow as tf

In [0]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        print(' Learning rate: {:10.9f}'.format(self.model.optimizer._decayed_lr(tf.float32)))

In [11]:
print(tf.__version__)

2.2.0


In [14]:
from __future__ import print_function
from tensorflow import keras # расскоментируйте эту строку, чтобы начать обучение
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import os
#import tensorflow as tf

# установка параметров нейросети
batch_size = 512
num_classes = 10
epochs = 80
l_2_reg = 1e-5
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
#opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3, decay_steps=1000, decay_rate=0.6)

#opt= keras.optimizers.Adam(learning_rate=lr_schedule)
opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])



x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 - 0.5
x_test /= 255 - 0.5

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры
Использование data augmentation в реальном времени
Epoch 1/80
98/98 [==============================] - 35s 359ms/step - loss: 2.0242 - accuracy: 0.3389 - val_loss: 3.3975 - val_accuracy: 0.1000
Epoch 2/80
98/98 [==============================] - 34s 342ms/step - loss: 1.5113 - accuracy: 0.4663 - val_loss: 2.9516 - val_accuracy: 0.1161
Epoch 3/80
98/98 [==============================] - 34s 344ms/step - loss: 1.3054 - accuracy: 0.5354 - val_loss: 2.8125 - val_accuracy: 0.2044
Epoch 4/80
98/98 [==============================] - 35s 353ms/step - loss: 1.1498 - accuracy: 0.5913 - val_loss: 2.2615 - val_accuracy: 0.2631
Epoch 5/80
98/98 [==============================] - 34s 346ms/step - loss: 1.0465 - accuracy: 0.6301 - val_loss: 1.3073 - val_accuracy: 0.5432
Epoch 6/80
98/98 [==============================] - 34s 345ms/step - loss: 0.9797 - accuracy: 0.6539 - val_loss: 0.8839 - val_accuracy: 0.6909
Epoch 

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


10000/10000 [==============================] - 2s 219us/step
Test loss: 0.4515459008216858
Test accuracy: 0.8497999906539917


In [1]:
# Задание 1
# 1. увеличение числа эпох до 30 повысило точность, но при этом алгоритм не успевал выходить на плато;
# 2. learning_rate был увеличен в 10 раз с 0.0001 до 0.001, что увеличило скорость сходимости алгоритма. При 
# этом алгоритм за 30 эпох успевал выйти на плато;
# 3. Adam увеличил скорость сходимости алгоритма, так как он позволяет быстрее выходить из седловых точек;
# 4. добавление BatchNormalization() увеличило скорость сходимости алгоритма, так как данные приводятся к одному масштабу;
# 5. удаление dropout слоев привело к переобучению: на тренировочных данных точность достигла 100% а на тестовых снизилась;
# 6. применеие l_2 регуляризации не дало улучшений, видимо потому что dropout хорошо боролся с переобучением;
# 7. применение learning rate sheduling повысило точность алгоритма, так как меньший шаг позволяет глубже погрузиться в 
# локальный минимум;
# 8. увеличение размера батча уменьшает время, затрачиваемое на прохождение конкретной эпохи, но увеличивает требования 
# к объему оперативной памяти;
# 9. добавление еще одного блока из 2-х сверточных слоев повысило точность сети (с одновременным уменьшением размера 
# полносвязного слоя), так как сверточная часть стала выявлять более абстрактные признаки;
# 10. использование аугументации повысило точность и уменьшило переобучение, так как данные стали более разнообразными

In [2]:
# Задание 2
# 1. если бы данная нейронная сеть работала с MNIST, то ее сложность была бы избыточной, так как входные данные 
# относительно просты. Необходимо уменьшить количество слоев и фильтров, так как паттерны в изображениях менее разнообразны,
# что повысит скорость работы и уменьшит вероятность переобучения;
# 2. чтобы данная нейронная сеть работала с CIFAR-100, необходимо параметру num_classes присвоить значение 100, чтобы
# последний полносвязный слой имел 100 выходов. Возможно потребуется больше фильтров, так как чем больше классов, тем больше
# возможных высокоабстрактных признаков можно извлечь из изображений
# 3. чтобы данная сеть работала с IMAGENET нужно провести больше всего изменений:
# увеличить число слоев и фильтров, так как входные изображения имеют большее разрешение а также большее разнообразие;
# параметру num_classes необходимо присвоить значение, равное количеству классов в датасете.